# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import string
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import joblib

from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
# from nltk import pos_tag, ne_chunk
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
def load_data(db, table_name):    
    engine = create_engine(db)
    df = pd.read_sql_table(con=engine, table_name=table_name)
    
    X = df[['message']]
    y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]
    
    return X, y

X, y = load_data('sqlite:///../data/DisasterResponse.db', 'cleaned_data')

display(X.head(5))
display(y.head(5))

,message
0,Weather update - a cold front from Cuba that c...
1,Is the Hurricane over or is it not over
2,Looking for someone but no name
3,UN reports Leogane 80-90 destroyed. Only Hospi...
4,"says: west side of Haiti, rest of the country ..."


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
y.shape

(26345, 36)

### 2. Write a tokenization function to process your text data

In [4]:
def is_stop_punc(token):
    """
    Returns True if the word is a punctuation or a stop word.
    """
    if token in string.punctuation:
        return True
    if token in stopwords.words("english"): 
        return True
    return False

def tokenize(text, stem_or_lem='stem'):
    """
    Text to lower, tokenise
    remove stop word 
    stemming and lemmatisation
    https://stackoverflow.com/questions/10554052/what-are-the-major-differences-and-benefits-of-porter-and-lancaster-stemming-alg
    Why snowball?    https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
    >>> tokenize("This is a test for the tokeniser to see how effective it is.")
    """
    text = text.lower()
    
    tokens = nltk.word_tokenize(text)
    tokens_clean = [token for token in tokens if not is_stop_punc(token)]
    
    if stem_or_lem.lower() == 'stem':
        stemmer = SnowballStemmer('english')
        result = [stemmer.stem(token) for token in tokens_clean]
    elif stem_or_lem.lower() == 'lem':
        lemmatiser = WordNetLemmatizer()
        result = [lemmatiser.lemmatize(token) for token in tokens_clean]
    else:
        raise ValueError('stem or lem. Could not understand \'%s\' ?' % stem_or_lem)
    
    return result

In [5]:
stemmed = tokenize("This is a test for the tokeniser to see how effective it is. !?daf")
lemmed  = tokenize("This is a test for the tokeniser to see how effective it is. !?daf", stem_or_lem='lem')
print(stemmed)
print(lemmed)

['test', 'tokenis', 'see', 'effect', 'daf']
['test', 'tokeniser', 'see', 'effective', 'daf']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()
    
    return {'Labels': labels,
            'Confusion Matrix': confusion_mat,
            'Accuracy': accuracy}

In [7]:
X, y = load_data('sqlite:///disaster_response.db', 'cleaned_data')
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = RandomForestClassifier()
m_clf = MultiOutputClassifier(estimator=clf)

In [9]:
# train classifier
print('X_train')
print(type(X_train.iloc[:, 0].values))
print(X_train.iloc[:, 0].values.shape)

X_train
<class 'numpy.ndarray'>
(19758,)


In [10]:
X_train_counts = vect.fit_transform(X_train.iloc[:, 0].values)
print('X_train_counts')
print(type(X_train_counts))
print(X_train_counts.shape)

# # view token vocabulary and counts
# vect.vocabulary_

X_train_counts
<class 'scipy.sparse.csr.csr_matrix'>
(19758, 26626)


In [11]:
X_train_tfidf = tfidf.fit_transform(X_train_counts)
print('X_train_tfidf')
print(type(X_train_tfidf))
print(X_train_tfidf.shape)

X_train_tfidf
<class 'scipy.sparse.csr.csr_matrix'>
(19758, 26626)


In [12]:
print(y_train.values.shape)
print(type(y_train.values))

m_clf.fit(X_train_tfidf, y_train.values)

(19758, 36)
<class 'numpy.ndarray'>


MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1)

In [13]:
# predict on test data
X_test_counts = vect.transform(X_test.iloc[:, 0].values)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = m_clf.predict(X_test_tfidf)

In [14]:
# display results
indv_results = {}

for i, classification in enumerate(y_test.columns):
    print(classification)
    indv_results[classification] = display_results(y_test.values[:, i], y_pred[:, i])
    print(indv_results[classification])

related
{'Labels': array([ 0.,  1.,  2.]), 'Confusion Matrix': array([[ 222, 1352,   10],
       [ 436, 4493,   26],
       [   5,   41,    2]]), 'Accuracy': 0.71610748443904659}
request
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[5358,  125],
       [1015,   89]]), 'Accuracy': 0.82693183543342952}
offer
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6555,    1],
       [  31,    0]]), 'Accuracy': 0.99514194625778052}
aid_related
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[3164,  747],
       [2106,  570]]), 'Accuracy': 0.56687414604524067}
medical_help
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6029,   34],
       [ 521,    3]]), 'Accuracy': 0.91574313040838018}
medical_products
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6244,   23],
       [ 316,    4]]), 'Accuracy': 0.94853499316836187}
search_and_rescue
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6405,   12],
       [ 169,    1]]), 'Accuracy': 0.9725

In [15]:
indv_reports = {}

for i, classification in enumerate(y_test.columns):
    print(classification)
    indv_reports[classification] = classification_report(y_test.values[:, i], y_pred[:, i])
    print(indv_reports[classification])

related
             precision    recall  f1-score   support

        0.0       0.33      0.14      0.20      1584
        1.0       0.76      0.91      0.83      4955
        2.0       0.05      0.04      0.05        48

avg / total       0.66      0.72      0.67      6587

request
             precision    recall  f1-score   support

        0.0       0.84      0.98      0.90      5483
        1.0       0.42      0.08      0.14      1104

avg / total       0.77      0.83      0.77      6587

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6556
        1.0       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6587

aid_related
             precision    recall  f1-score   support

        0.0       0.60      0.81      0.69      3911
        1.0       0.43      0.21      0.29      2676

avg / total       0.53      0.57      0.53      6587

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
# word2vec 
# Continuous Bag of Words
# Skip-gram : one hot encode and pasa to neural networks

# or GloVe: Global Vectors


# collection of documents is a corpus
# vocabulary
# term frequency: How many times does the word appear in a document
# document frequency : How many time does the word appear in corpus
# tfidf (t, d, D) = count(t, d)/|d|  * log(|D|/ num of docs where t is present)

In [17]:
pipeline = Pipeline([
    ('tokenise', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('multi_class', MultiOutputClassifier(estimator=RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X, y = load_data('sqlite:///../data/DisasterResponse.db', 'cleaned_data')
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
pipeline.fit(X_train.iloc[:, 0].values, y_train.values)

Pipeline(memory=None,
     steps=[('tokenise', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        s...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
y_pred = pipeline.predict(X_test.iloc[:, 0].values)

In [21]:
# display results
indv_results = {}

for i, classification in enumerate(y_test.columns):
    print(classification)
    indv_results[classification] = display_results(y_test.values[:, i], y_pred[:, i])
    print(indv_results[classification])

related
{'Labels': array([ 0.,  1.,  2.]), 'Confusion Matrix': array([[ 216, 1345,   11],
       [ 443, 4494,   27],
       [   6,   43,    2]]), 'Accuracy': 0.71534841354182477}
request
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[5276,  158],
       [1066,   87]]), 'Accuracy': 0.81417944436010325}
offer
{'Labels': array([ 0.]), 'Confusion Matrix': array([[6558]]), 'Accuracy': 0.99559738879611359}
aid_related
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[3105,  796],
       [2098,  588]]), 'Accuracy': 0.56064976468802186}
medical_help
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6021,   41],
       [ 524,    1]]), 'Accuracy': 0.91422498861393653}
medical_products
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6235,   21],
       [ 330,    1]]), 'Accuracy': 0.94671322301502958}
search_and_rescue
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6404,    4],
       [ 179,    0]]), 'Accuracy': 0.97221800516168211}
security
{'Labels

In [22]:
indv_report = {}

for i, classification in enumerate(y_test.columns):
    print(classification)
    indv_report[classification] = classification_report(y_test.values[:, i], y_pred[:, i])
    print(indv_report[classification])

related
             precision    recall  f1-score   support

        0.0       0.32      0.14      0.19      1572
        1.0       0.76      0.91      0.83      4964
        2.0       0.05      0.04      0.04        51

avg / total       0.65      0.72      0.67      6587

request
             precision    recall  f1-score   support

        0.0       0.83      0.97      0.90      5434
        1.0       0.36      0.08      0.12      1153

avg / total       0.75      0.81      0.76      6587

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6558
        1.0       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6587

aid_related
             precision    recall  f1-score   support

        0.0       0.60      0.80      0.68      3901
        1.0       0.42      0.22      0.29      2686

avg / total       0.53      0.56      0.52      6587

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
parameters = parameters = {'multi_class__estimator__n_estimators': [1, 10],
                           # 'multi_class__estimator__criterion': ['gini', 'entropy'],
                           'multi_class__estimator__min_samples_split': [2, 10],
                          }

cv = GridSearchCV(pipeline, parameters)
cv_fit = cv.fit(X_train.iloc[:, 0].values, y_train.values)

In [28]:
# Save the resulat
filename = 'grid_search.sav'
dump(cv, filename)

['grid_search.sav']

In [29]:
print("Best score: %0.3f" % cv.best_score_)
print("Best parameters set:")
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.138
Best parameters set:
	multi_class__estimator__min_samples_split: 2
	multi_class__estimator__n_estimators: 10


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
y_pred = cv.predict(X_test.iloc[:, 0].values)

for i, classification in enumerate(y_test.columns):
    print(classification)
    indv_results[classification] = display_results(y_test.values[:, i], y_pred[:, i])
    print(indv_results[classification])

related
{'Labels': array([ 0.,  1.,  2.]), 'Confusion Matrix': array([[ 213, 1352,    7],
       [ 439, 4506,   19],
       [  10,   41,    0]]), 'Accuracy': 0.71641111279793535}
request
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[5262,  172],
       [1058,   95]]), 'Accuracy': 0.81326855928343711}
offer
{'Labels': array([ 0.]), 'Confusion Matrix': array([[6558]]), 'Accuracy': 0.99559738879611359}
aid_related
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[3098,  803],
       [2055,  631]]), 'Accuracy': 0.56611507514801884}
medical_help
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6029,   33],
       [ 525,    0]]), 'Accuracy': 0.91528768787004711}
medical_products
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6238,   18],
       [ 330,    1]]), 'Accuracy': 0.94716866555336265}
search_and_rescue
{'Labels': array([ 0.,  1.]), 'Confusion Matrix': array([[6403,    5],
       [ 179,    0]]), 'Accuracy': 0.97206619098223779}
security
{'Labels

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Other features?
# Other methods: OneVsRestClassifier?
# SGDClassifier


### 9. Export your model as a pickle file

In [25]:
# Save model
filename = 'finalized_model.sav'
dump(pipeline, filename)

0.192500379535


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.